In [1]:
# install pipenv requirements
!pipenv install
import dotenv
import os

# load environment variables
dotenv.load_dotenv()
SSH_USERNAME = os.getenv("SSH_USERNAME")
SSH_HOST = os.getenv("SSH_HOST")
SSH_PORT = os.getenv("SSH_PORT")
SSH_PASSWORD = os.getenv("SSH_PASSWORD")
NS3DIR = os.getenv("NS3DIR")

Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing dependencies from Pipfile.lock (7bd759)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [2]:
from wifi_experiment.utlis import *

# connect to remote host
host = remote_host(host=SSH_HOST, port=SSH_PORT, username=SSH_USERNAME, password=SSH_PASSWORD, ns3dir=NS3DIR)

In [3]:
output = host.exec(f"{host.ns3dir}/ns3")

AttributeError: 'remote_host' object has no attribute 'execute'

In [4]:
# setup example experiment
propModel = propagationModels[0]
run = experiment_run(host=host, propModel=propModel, simulationTime_s=10, distance_m=10, remote_csv_path=f"{host.ns3dir}/sample.csv", local_csv_path="sample.csv")
run.run()

KeyboardInterrupt: 

In [9]:
output = host.exec("source ~/.bashrc; echo $NS3DIR")
print(output)